In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Read the Excel file into a Pandas dataframe
df = pd.read_excel("Input.xlsx")

# Loop through each row of the dataframe
for index, row in df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    
    # Use requests library to fetch the HTML content of the URL
    response = requests.get(url)
    
    # Use beautifulsoup4 to parse the HTML content and extract the article title and text
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('title').text.strip()
    article_text = ''
    for paragraph in soup.find_all('p'):
        article_text += paragraph.text
    
    # Write the extracted article text to a text file with url_id as filename
    with open(f"{url_id}.txt", "w", encoding='utf-8') as text_file:
        text_file.write(f"{title}\n\n{article_text}")

In [14]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [25]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\peridynamics\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peridynamics\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
def clean_text(text):
    stop_words = list(stopwords.words('english'))
    with open("StopWords/StopWords_Names.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Geographic.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_GenericLong.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Generic.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_DatesandNumbers.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Currencies.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Auditor.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words and word.isalpha()]
    return " ".join(filtered_text)

In [16]:
def create_word_dict():
    positive_words = []
    negative_words = []
    with open("MasterDictionary/positive-words.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                positive_words.append(line.strip())
    with open("MasterDictionary/negative-words.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                negative_words.append(line.strip())
    stop_words = list(stopwords.words('english'))
    with open("StopWords/StopWords_Names.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Geographic.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_GenericLong.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Generic.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_DatesandNumbers.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Currencies.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    with open("StopWords/StopWords_Auditor.txt",'r') as f:
        for line in f:
            if not line.startswith(';'):
                stop_words.append(line.strip())
    positive_words = [word for word in positive_words if word not in stop_words]
    negative_words = [word for word in negative_words if word not in stop_words]
    return positive_words, negative_words

In [17]:
def sentimental_analysis(text):
    positive_words, negative_words = create_word_dict()
    word_tokens = word_tokenize(text)
    cleaned_text = [word for word in word_tokens if word.lower() not in stop_words and word.isalpha()]
    positive_score = len([word for word in cleaned_text if word in positive_words])
    negative_score = len([word for word in cleaned_text if word in negative_words])
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / ((len(cleaned_text)) + 0.000001)
    return positive_score, negative_score, polarity_score, subjectivity_score

In [30]:
def syllables_per_word(word_tokens):
    vowels = ['a', 'e', 'i', 'o', 'u']

    syllable_counts = []

    for word in word_tokens:
        count = 0
        for i in range(len(word)):
            if word[i].lower() in vowels:
                count += 1
                if i == len(word) - 1 and (word.endswith("es") or word.endswith("ed")):
                    count -= 1
        syllable_counts.append(count)
        
    return len(syllable_counts)

In [41]:
def readability_analysis(text):
    word_tokens = word_tokenize(text)
    sentence_tokens = sent_tokenize(text)
    word_count = len([word for word in word_tokens if word.isalpha()])
    total_sentences = len(sentence_tokens)
    avg_sentence_length = word_count / total_sentences
    avg_number_of_words_per_sentence = word_count / total_sentences
    complex_word_count = len([word for word in word_tokens if len(word) > 2 and word.isalpha()])
    percentage_complex_words = (complex_word_count / word_count) * 100
    syllable_per_word = syllables_per_word(word_tokens)
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_word_length = sum([len(word) for word in word_tokens])/len(word_tokens)
    return avg_sentence_length, percentage_complex_words, fog_index, avg_number_of_words_per_sentence, complex_word_count, word_count, syllable_per_word, avg_word_length

In [19]:
def personal_pronouns(text):
    personal_pronoun_list = ["I", "we", "my", "ours", "us"]
    country_name = "US"
    word_tokens = word_tokenize(text)
    personal_pronouns = len([word for word in word_tokens if word.lower() in personal_pronoun_list and word.lower() != country_name.lower()])
    return personal_pronouns

In [42]:
for num in range(37, 151):
    with open(f"{num}.0.txt", "r", encoding='utf-8') as file:
        contents = file.read()
        clean_text(contents)
        create_word_dict()
        print(f"For the file: {num}.0")
        print()
        print("Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score")
        print(sentimental_analysis(contents))
        print()
        print("Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length")
        print(readability_analysis(contents))
        print()
        print("Personal Pronouns:")
        print(personal_pronouns(contents))
        print("--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

For the file: 37.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(63, 31, 0.34042552829334544, 0.08842897451700002)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(32.264150943396224, 81.69590643274853, 45.58402295045791, 32.264150943396224, 1397, 1710, 1956, 5.284253578732106)

Personal Pronouns:
0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 38.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(55, 35, 0.22222221975308645, 0.1255230123772343)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Lengt

For the file: 49.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(26, 24, 0.03999999920000002, 0.07278020367863143)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(17.984615384615385, 80.06843455945253, 39.221219977627165, 17.984615384615385, 936, 1169, 1383, 4.5531453362255965)

Personal Pronouns:
3
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 50.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(54, 23, 0.40259739736886496, 0.10225763598637765)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Len

For the file: 62.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(3, 1, 0.49999987500003124, 0.018433179638556774)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(28.46153846153846, 83.51351351351352, 44.79002079002079, 28.46153846153846, 309, 370, 412, 4.645631067961165)

Personal Pronouns:
0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 63.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(19, 7, 0.4615384437869829, 0.061611374261584424)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(42.

For the file: 74.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(33, 24, 0.1578947340720222, 0.07529722579220974)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(19.732394366197184, 79.87152034261243, 39.84156588352385, 19.732394366197184, 1119, 1401, 1658, 4.393244873341375)

Personal Pronouns:
13
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 75.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(23, 5, 0.64285711989796, 0.07909604497430496)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(

For the file: 86.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(23, 17, 0.1499999962500001, 0.09732360073643893)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(39.611111111111114, 81.7671809256662, 48.55131681471093, 39.611111111111114, 583, 713, 812, 4.911330049261084)

Personal Pronouns:
0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 87.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(20, 28, -0.16666666319444454, 0.07741935471383975)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(

For the file: 98.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(34, 45, -0.13924050456657588, 0.12783171500350854)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(17.691176470588236, 79.7173732335827, 38.96341988166838, 17.691176470588236, 959, 1203, 1336, 4.457335329341317)

Personal Pronouns:
31
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 99.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(22, 26, -0.08333333159722227, 0.0831889080014057)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Leng

For the file: 111.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(12, 27, -0.384615374753452, 0.0724907061849615)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(26.210526315789473, 85.2409638554217, 44.58059606848447, 26.210526315789473, 849, 996, 1124, 4.9564056939501775)

Personal Pronouns:
0
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 112.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(24, 47, -0.3239436574092443, 0.13027522911876105)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length

For the file: 123.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(36, 77, -0.3628318551961783, 0.11970338970370403)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(42.375, 80.64896755162242, 49.20958702064897, 42.375, 1367, 1695, 1936, 4.785640495867769)

Personal Pronouns:
10
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 124.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(20, 41, -0.34426228943832315, 0.11641221351829731)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(24.2564102564102

For the file: 135.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(6, 7, -0.07692307100591762, 0.07514450823615891)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(23.571428571428573, 77.57575757575758, 40.45887445887446, 23.571428571428573, 256, 330, 382, 4.227748691099476)

Personal Pronouns:
1
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 136.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(0, 16, -0.9999999375000038, 0.09876543148910227)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length


For the file: 148.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(26, 43, -0.24637680802352452, 0.10832025100734655)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length
(18.18032786885246, 76.91614066726781, 38.03858741444811, 18.18032786885246, 853, 1109, 1255, 4.673306772908367)

Personal Pronouns:
2
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
For the file: 149.0

Set: Positive Score, Negative Score, Polarity Score, Subjectivity Score
(28, 3, 0.8064515868886585, 0.10064935032256704)

Set: Avg Sentence Length, Percentage Complex Words, Fog Index, Avg Number of Words per Sentence, Complex Word Count, Word Count, Syllable per Word, Avg Word Length

In [40]:
columns = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']
output_df = pd.DataFrame(columns=columns)
output_df.to_excel('Output.xlsx', index=False)

In [64]:
from openpyxl import load_workbook

book = load_workbook('Output.xlsx')

writer = pd.ExcelWriter('Output.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
worksheet_name = 'output'
writer.sheets[worksheet_name] = book.active

new_data = pd.DataFrame({'URL_ID': [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150],
                         'URL': ['https://insights.blackcoffer.com/ai-in-healthcare-to-improve-patient-outcomes/', 'https://insights.blackcoffer.com/what-if-the-creation-is-taking-over-the-creator/', 'https://insights.blackcoffer.com/what-jobs-will-robots-take-from-humans-in-the-future/', 'https://insights.blackcoffer.com/will-machine-replace-the-human-in-the-future-of-work/', 'https://insights.blackcoffer.com/will-ai-replace-us-or-work-with-us/', 'https://insights.blackcoffer.com/man-and-machines-together-machines-are-more-diligent-than-humans-blackcoffe/', 'https://insights.blackcoffer.com/in-future-or-in-upcoming-years-humans-and-machines-are-going-to-work-together-in-every-field-of-work/', 'https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/', 'https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/', 'https://insights.blackcoffer.com/deep-learning-impact-on-areas-of-e-learning/', 'https://insights.blackcoffer.com/how-to-protect-future-data-and-its-privacy-blackcoffer/', 'https://insights.blackcoffer.com/how-machines-ai-automations-and-robo-human-are-effective-in-finance-and-banking/', 'https://insights.blackcoffer.com/ai-human-robotics-machine-future-planet-blackcoffer-thinking-jobs-workplace/', 'https://insights.blackcoffer.com/how-ai-will-change-the-world-blackcoffer/', 'https://insights.blackcoffer.com/future-of-work-how-ai-has-entered-the-workplace/', 'https://insights.blackcoffer.com/ai-tool-alexa-google-assistant-finance-banking-tool-future/', 'https://insights.blackcoffer.com/ai-healthcare-revolution-ml-technology-algorithm-google-analytics-industrialrevolution/', 'https://insights.blackcoffer.com/all-you-need-to-know-about-online-marketing/', 'https://insights.blackcoffer.com/evolution-of-advertising-industry/', 'https://insights.blackcoffer.com/how-data-analytics-can-help-your-business-respond-to-the-impact-of-covid-19/', 'https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/', 'https://insights.blackcoffer.com/environmental-impact-of-the-covid-19-pandemic-lesson-for-the-future/', 'https://insights.blackcoffer.com/how-data-analytics-and-ai-are-used-to-halt-the-covid-19-pandemic/', 'https://insights.blackcoffer.com/difference-between-artificial-intelligence-machine-learning-statistics-and-data-mining/', 'https://insights.blackcoffer.com/how-python-became-the-first-choice-for-data-science/', 'https://insights.blackcoffer.com/how-google-fit-measure-heart-and-respiratory-rates-using-a-phone/', 'https://insights.blackcoffer.com/what-is-the-future-of-mobile-apps/', 'https://insights.blackcoffer.com/impact-of-ai-in-health-and-medicine/', 'https://insights.blackcoffer.com/telemedicine-what-patients-like-and-dislike-about-it/', 'https://insights.blackcoffer.com/how-we-forecast-future-technologies/', 'https://insights.blackcoffer.com/can-robots-tackle-late-life-loneliness/', 'https://insights.blackcoffer.com/embedding-care-robots-into-society-socio-technical-considerations/', 'https://insights.blackcoffer.com/management-challenges-for-future-digitalization-of-healthcare-services/', 'https://insights.blackcoffer.com/are-we-any-closer-to-preventing-a-nuclear-holocaust/', 'https://insights.blackcoffer.com/will-technology-eliminate-the-need-for-animal-testing-in-drug-development/', 'https://insights.blackcoffer.com/will-we-ever-understand-the-nature-of-consciousness/', 'https://insights.blackcoffer.com/will-we-ever-colonize-outer-space/', 'https://insights.blackcoffer.com/what-is-the-chance-homo-sapiens-will-survive-for-the-next-500-years/', 'https://insights.blackcoffer.com/why-does-your-business-need-a-chatbot/', 'https://insights.blackcoffer.com/how-you-lead-a-project-or-a-team-without-any-technical-expertise/', 'https://insights.blackcoffer.com/can-you-be-great-leader-without-technical-expertise/', 'https://insights.blackcoffer.com/how-does-artificial-intelligence-affect-the-environment/', 'https://insights.blackcoffer.com/how-to-overcome-your-fear-of-making-mistakes-2/', 'https://insights.blackcoffer.com/is-perfection-the-greatest-enemy-of-productivity/', 'https://insights.blackcoffer.com/global-financial-crisis-2008-causes-effects-and-its-solution/', 'https://insights.blackcoffer.com/gender-diversity-and-equality-in-the-tech-industry/', 'https://insights.blackcoffer.com/how-to-overcome-your-fear-of-making-mistakes/', 'https://insights.blackcoffer.com/how-small-business-can-survive-the-coronavirus-crisis/', 'https://insights.blackcoffer.com/impacts-of-covid-19-on-vegetable-vendors-and-food-stalls/', 'https://insights.blackcoffer.com/impacts-of-covid-19-on-vegetable-vendors/', 'https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-tourism-aviation-industries/', 'https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-sports-events-around-the-world/', 'https://insights.blackcoffer.com/changing-landscape-and-emerging-trends-in-the-indian-it-ites-industry/', 'https://insights.blackcoffer.com/online-gaming-adolescent-online-gaming-effects-demotivated-depression-musculoskeletal-and-psychosomatic-symptoms/', 'https://insights.blackcoffer.com/human-rights-outlook/', 'https://insights.blackcoffer.com/how-voice-search-makes-your-business-a-successful-business/', 'https://insights.blackcoffer.com/how-the-covid-19-crisis-is-redefining-jobs-and-services/', 'https://insights.blackcoffer.com/how-to-increase-social-media-engagement-for-marketers/', 'https://insights.blackcoffer.com/impacts-of-covid-19-on-streets-sides-food-stalls/', 'https://insights.blackcoffer.com/coronavirus-impact-on-energy-markets-2/', 'https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-5/', 'https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-4/', 'https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work-2/', 'https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work-3/', 'https://insights.blackcoffer.com/travel-and-tourism-outlook/', 'https://insights.blackcoffer.com/gaming-disorder-and-effects-of-gaming-on-health/', 'https://insights.blackcoffer.com/what-is-the-repercussion-of-the-environment-due-to-the-covid-19-pandemic-situation/', 'https://insights.blackcoffer.com/what-is-the-repercussion-of-the-environment-due-to-the-covid-19-pandemic-situation-2/', 'https://insights.blackcoffer.com/impact-of-covid-19-pandemic-on-office-space-and-co-working-industries/', 'https://insights.blackcoffer.com/contribution-of-handicrafts-visual-arts-literature-in-the-indian-economy/', 'https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/', 'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/', 'https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis/', 'https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding/', 'https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-2/', 'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-3/', 'https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-3/', 'https://insights.blackcoffer.com/estimating-the-impact-of-covid-19-on-the-world-of-work/', 'https://insights.blackcoffer.com/covid-19-how-have-countries-been-responding-2/', 'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-4/', 'https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-2/', 'https://insights.blackcoffer.com/lessons-from-the-past-some-key-learnings-relevant-to-the-coronavirus-crisis-3/', 'https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry-4/', 'https://insights.blackcoffer.com/why-scams-like-nirav-modi-happen-with-indian-banks/', 'https://insights.blackcoffer.com/impact-of-covid-19-on-the-global-economy/', 'https://insights.blackcoffer.com/impact-of-covid-19coronavirus-on-the-indian-economy-2/', 'https://insights.blackcoffer.com/impact-of-covid-19-on-the-global-economy-2/', 'https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy-3/', 'https://insights.blackcoffer.com/should-celebrities-be-allowed-to-join-politics/', 'https://insights.blackcoffer.com/how-prepared-is-india-to-tackle-a-possible-covid-19-outbreak/', 'https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work/', 'https://insights.blackcoffer.com/controversy-as-a-marketing-strategy/', 'https://insights.blackcoffer.com/coronavirus-impact-on-the-hospitality-industry/', 'https://insights.blackcoffer.com/coronavirus-impact-on-energy-markets/', 'https://insights.blackcoffer.com/what-are-the-key-policies-that-will-mitigate-the-impacts-of-covid-19-on-the-world-of-work/', 'https://insights.blackcoffer.com/marketing-drives-results-with-a-focus-on-problems/', 'https://insights.blackcoffer.com/continued-demand-for-sustainability/', 'https://insights.blackcoffer.com/coronavirus-disease-covid-19-effect-the-impact-and-role-of-mass-media-during-the-pandemic/', 'https://insights.blackcoffer.com/should-people-wear-fabric-gloves-seeking-evidence-regarding-the-differential-transfer-of-covid-19-or-coronaviruses-generally-between-surfaces/', 'https://insights.blackcoffer.com/why-is-there-a-severe-immunological-and-inflammatory-explosion-in-those-affected-by-sarms-covid-19/', 'https://insights.blackcoffer.com/what-do-you-think-is-the-lesson-or-lessons-to-be-learned-with-covid-19/', 'https://insights.blackcoffer.com/coronavirus-the-unexpected-challenge-for-the-european-union/', 'https://insights.blackcoffer.com/industrial-revolution-4-0-pros-and-cons/', 'https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy/', 'https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-indian-economy-2/', 'https://insights.blackcoffer.com/impact-of-covid-19coronavirus-on-the-indian-economy/', 'https://insights.blackcoffer.com/impact-of-covid-19-coronavirus-on-the-global-economy/', 'https://insights.blackcoffer.com/ensuring-growth-through-insurance-technology/', 'https://insights.blackcoffer.com/blockchain-in-fintech/', 'https://insights.blackcoffer.com/blockchain-for-payments/', 'https://insights.blackcoffer.com/the-future-of-investing/', 'https://insights.blackcoffer.com/big-data-analytics-in-healthcare/', 'https://insights.blackcoffer.com/business-analytics-in-the-healthcare-industry/', 'https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/'],
                         'POSITIVE SCORE': [63, 55, 65, 55, 46, 42, 25, 0, 35, 63, 44, 34, 26, 54, 57, 25, 80, 17, 14, 0, 0, 24, 17, 54, 38, 3, 19, 13, 15, 5, 22, 9, 32, 1, 9, 25, 39, 33, 23, 39, 28, 40, 22, 48, 17, 21, 21, 20, 8, 23, 20, 17, 34, 26, 3, 28, 32, 43, 28, 15, 16, 34, 22, 13, 4, 28, 7, 27, 20, 0, 11, 27, 28, 5, 12, 24, 16, 43, 17, 24, 31, 32, 8, 41, 33, 30, 36, 20, 55, 15, 35, 7, 5, 22, 79, 21, 23, 7, 6, 0, 4, 5, 4, 12, 5, 37, 5, 0, 11, 20, 33, 26, 28, 32],
                         'NEGATIVE SCORE': [31, 35, 32, 21, 22, 23, 12, 0, 13, 38, 58, 19, 24, 23, 21, 0, 39, 0, 6, 3, 0, 66, 14, 37, 3, 1, 7, 8, 8, 0, 34, 2, 10, 6, 25, 45, 21, 24, 5, 8, 6, 14, 66, 22, 25, 21, 72, 8, 6, 17, 28, 10, 41, 61, 1, 4, 34, 14, 30, 41, 40, 45, 26, 31, 3, 44, 24, 52, 34, 0, 3, 51, 21, 33, 27, 47, 21, 45, 38, 25, 63, 52, 14, 87, 60, 63, 77, 41, 17, 23, 83, 15, 6, 54, 30, 20, 18, 10, 7, 16, 7, 10, 2, 51, 19, 63, 24, 0, 11, 26, 11, 43, 3, 38],
                         'POLARITY SCORE': [0.34042552829334544, 0.22222221975308645, 0.3402061820597301, 0.44736841516620507, 0.3529411712802769, 0.292307687810651, 0.35135134185536915, 0.0, 0.45833332378472247, 0.24752475002450744, -0.13725490061514803, 0.2830188625845498, 0.03999999920000002, 0.40259739736886496, 0.46153845562130186, 0.9999999600000016, 0.3445378122307747, 0.999999941176474, 0.399999980000001, -0.9999996666667778, 0.0, -0.46666666148148156, 0.09677419042663901, 0.18681318476029468, 0.8536585157644265, 0.49999987500003124, 0.4615384437869829, 0.23809522675737013, 0.3043478128544429, 0.99999980000004, -0.21428571045918376, 0.636363578512402, 0.5238095113378688, -0.7142856122449125, -0.47058822145328766, -0.2857142816326531, 0.2999999950000001, 0.1578947340720222, 0.64285711989796, 0.6595744540516074, 0.6470588044982705, 0.48148147256515794, -0.4999999943181819, 0.37142856612244907, -0.1904761859410432, 0.0, -0.5483870908775582, 0.4285714132653067, 0.14285713265306196, 0.1499999962500001, -0.16666666319444454, 0.2592592496570648, -0.09333333208888892, -0.402298845950588, 0.49999987500003124, 0.7499999765625008, -0.03030302984389349, 0.5087719208987382, -0.03448275802615935, -0.4642857059948981, -0.42857142091836753, -0.13924050456657588, -0.08333333159722227, -0.40909089979338864, 0.1428571224489825, -0.2222222191358025, -0.5483870790842877, -0.3164556921967634, -0.25925925445816195, 0.0, 0.5714285306122479, -0.3076923037475346, 0.14285713994169102, -0.7368420858725767, -0.384615374753452, -0.3239436574092443, -0.1351351314828343, -0.02272727246900827, -0.3818181748760332, -0.020408162848813003, -0.34042552829334544, -0.238095235260771, -0.27272726033057904, -0.35937499719238286, -0.2903225775234132, -0.35483870586194943, -0.3628318551961783, -0.34426228943832315, 0.5277777704475309, -0.21052631024930765, -0.40677965756966394, -0.36363634710743875, -0.09090908264462885, -0.42105262603878124, 0.4495412802794378, 0.024390243307555043, 0.12195121653777521, -0.17647057785467188, -0.07692307100591762, -0.9999999375000038, -0.2727272479338866, -0.3333333111111126, 0.33333327777778704, -0.6190476092214665, -0.5833333090277788, -0.2599999974, -0.655172391200952, 0.0, 0.0, -0.13043477977315696, 0.4999999886363639, -0.24637680802352452, 0.8064515868886585, -0.08571428448979594],
                         'SUBJECTIVITY SCORE': [0.08842897451700002, 0.1255230123772343, 0.10167714874038035, 0.09212121200955005, 0.07649043860912212, 0.10333863258610711, 0.09560723489507174, 0.0, 0.13008130046048427, 0.09042076983847737, 0.09970674477057014, 0.0817901233305708, 0.07278020367863143, 0.10225763598637765, 0.09725685773409369, 0.08650519001209277, 0.13296089370618894, 0.04450261768454812, 0.05291005277007923, 0.029999999700000005, 0.0, 0.12499999982638889, 0.08179419503484381, 0.09784946226037693, 0.11884057936567947, 0.018433179638556774, 0.061611374261584424, 0.08076923045857988, 0.12365591331367788, 0.05494505434126314, 0.14177215153981734, 0.1134020606865767, 0.11260053589115138, 0.09459459331628928, 0.1888888878395062, 0.07592190881136453, 0.07142857134353742, 0.07529722579220974, 0.07909604497430496, 0.07241910620582573, 0.14782608631379962, 0.06301050167676721, 0.18763326186005702, 0.13084112125076425, 0.07636363622479339, 0.07381370813038013, 0.19175257692422149, 0.08211143670935063, 0.08974358916831032, 0.09732360073643893, 0.07741935471383975, 0.01461829993794353, 0.0867052022119015, 0.10151691936812494, 0.06060605968778698, 0.07142857126913266, 0.07525655635660598, 0.06499429867161426, 0.10086956504196598, 0.06229143485840775, 0.11359026346127736, 0.12783171500350854, 0.0831889080014057, 0.0706260030969085, 0.07954545364152894, 0.12435233139144675, 0.09509202424818397, 0.12908496710933828, 0.10953346833766031, 0.0, 0.04946996448950543, 0.16810344791357015, 0.15408804982991178, 0.07116104855587818, 0.0724907061849615, 0.13027522911876105, 0.08525345602476163, 0.11013767195226824, 0.06707317064991077, 0.11290322554630593, 0.10398230076993108, 0.14237288111462224, 0.12429378460850969, 0.10314262683066669, 0.12739726009945582, 0.10508474564397204, 0.11970338970370403, 0.11641221351829731, 0.07792207783774667, 0.08775981503981567, 0.1676136361255488, 0.1466666656888889, 0.04382470102061872, 0.07810894133801753, 0.11695278957408499, 0.05710306398732164, 0.10249999974375, 0.09189189139517896, 0.07514450823615891, 0.09876543148910227, 0.09090909015777611, 0.12605041910882, 0.06382978655500227, 0.126252504757009, 0.09999999958333333, 0.10526315778393353, 0.09148264955368249, 0.0, 0.05714285699443414, 0.09745762691216604, 0.07432432419877648, 0.10832025100734655, 0.10064935032256704, 0.12173913022306239],
                         'AVG SENTENCE LENGTH': [32.26415094, 20.39705882, 24.36764706, 20.27631579, 24.62686567, 26.43478261, 17.43902439, 5, 23.2, 30.42028986, 20.81818182, 27.80487805, 17.98461538, 49.125, 28.32692308, 27.33333333, 225.5, 28.2, 31.57142857, 85.5, 5, 31.45238095, 29, 21.10227273, 22.21428571, 28.46153846, 42.88888889, 34.69230769, 16.60869565, 17.33333333, 14.83636364, 29, 31.9, 25.5, 18.28571429, 32.82352941, 36.67391304, 19.73239437, 15.53703704, 22.61904762, 33.64285714, 36.11904762, 15.03076923, 19, 33.51724138, 25.625, 15.88888889, 28.39130435, 29.1, 39.61111111, 33.58823529, 18.47852761, 27.12903226, 29.5, 33.5, 25.15151515, 16.96190476, 22.74647887, 28.5, 35.79545455, 27.3030303, 17.69117647, 35.8, 40, 29.16666667, 20.75, 31.44444444, 24.06666667, 39.03703704, 48, 25.4, 43.8, 43.625, 39.58333333, 26.21052632, 22.53488372, 27.03448276, 24.51612903, 32.41860465, 84.5, 37.74418605, 18.9375, 30.4, 26.74025974, 28.28571429, 24.14925373, 42.375, 24.25641026, 31.39285714, 27.93103448, 26, 22.38461538, 28.86666667, 142.4545455, 26.20967742, 22.81034483, 27.25, 26.41666667, 23.57142857, 31.16666667, 37, 58.5, 20.71428571, 24.75, 41.36363636, 29.27118644, 21.74074074, 5, 25.67857143, 22, 26.15, 18.18032787, 34.125, 16.51612903],
                         'PERCENTAGE OF COMPLEX WORDS': [81.69590643, 77.86589762, 78.87748944, 76.83322518, 78.96969697, 79.52302632, 81.3986014, 100, 80.17241379, 75.98856598, 79.36681223, 81.66666667, 80.06843456, 76.84478372, 77.59674134, 78.04878049, 76.94013304, 81.13475177, 76.31975867, 84.21052632, 100, 82.43754731, 79.31034483, 77.86752827, 79.42122186, 83.51351351, 81.86528497, 78.27050998, 82.19895288, 75.64102564, 77.45098039, 81.6091954, 83.54231975, 81.04575163, 74.47916667, 78.19593787, 79.90515708, 79.87152034, 77.35399285, 75.36842105, 74.73460722, 78.37837838, 77.37973388, 78.58439201, 85.3909465, 79.02439024, 77.92207792, 80.55130168, 79.72508591, 81.76718093, 82.6619965, 84.56175299, 78.29964328, 82.48587571, 79.10447761, 82.65060241, 82.76249298, 79.75232198, 80.89668616, 80.95238095, 80.46614872, 79.71737323, 80.72625698, 79.51923077, 78.28571429, 77.84986098, 80.91872792, 79.77839335, 78.9373814, 83.33333333, 83.07086614, 80.3652968, 76.50429799, 85.36842105, 85.24096386, 81.52734778, 81.37755102, 79.01315789, 81.13342898, 76.68639053, 82.43992606, 78.05280528, 82.56578947, 83.68139874, 80.80808081, 81.14956737, 80.64896755, 79.06976744, 80.37542662, 80.24691358, 80.37676609, 75.94501718, 81.7551963, 80.47223995, 81.10769231, 80.72562358, 85.32110092, 81.70347003, 77.57575758, 72.19251337, 82.88288288, 77.35042735, 82.06896552, 81.81818182, 81.97802198, 79.61783439, 81.26064736, 100, 79.83310153, 79.54545455, 79.54110899, 76.91614067, 82.6007326, 78.41796875],
                         'FOG INDEX': [45.58402295, 39.30518258, 41.2980546, 38.84381639, 41.43862506, 42.38312357, 39.53505032, 42, 41.34896552, 42.56354234, 40.07399762, 43.78861789, 39.22121998, 50.38791349, 42.36946577, 42.15284553, 120.9760532, 43.73390071, 43.1564749, 67.88421053, 42, 45.55597131, 43.32413793, 39.5879204, 40.65420303, 44.79002079, 49.90166955, 45.18512707, 39.52305941, 37.18974359, 36.91493761, 44.24367816, 46.1769279, 42.61830065, 37.10595238, 44.40778691, 46.63162805, 39.84156588, 37.15641195, 39.19498747, 43.35098574, 45.7989704, 36.96420124, 39.03375681, 47.56327515, 41.8597561, 37.52438672, 43.57704241, 43.53003436, 48.55131681, 46.50009272, 41.21611224, 42.17147022, 44.79435028, 45.04179104, 43.12084702, 39.8897591, 40.99952034, 43.75867446, 46.6991342, 43.10767161, 38.96341988, 46.61050279, 47.80769231, 42.98095238, 39.43994439, 44.94526894, 41.53802401, 47.18976738, 52.53333333, 43.38834646, 49.66611872, 48.0517192, 49.98070175, 44.58059607, 41.6248926, 43.36481351, 41.41171477, 45.42081345, 64.47455621, 48.07364484, 38.79612211, 45.18631579, 44.16866339, 43.63751804, 42.11952844, 49.20958702, 41.33047108, 44.70731351, 43.27117923, 42.55070644, 39.33185303, 44.24874519, 89.17071416, 42.92694789, 41.41438736, 45.02844037, 43.24805468, 40.45887446, 41.34367201, 47.95315315, 54.34017094, 41.11330049, 42.62727273, 49.33666334, 43.55560833, 41.20055524, 42, 42.20466918, 40.61818182, 42.27644359, 38.03858741, 46.69029304, 37.97363911],
                         'AVG NUMBER OF WORDS PER SENTENCE': [32.26415094, 20.39705882, 24.36764706, 20.27631579, 24.62686567, 26.43478261, 17.43902439, 5, 23.2, 30.42028986, 20.81818182, 27.80487805, 17.98461538, 49.125, 28.32692308, 27.33333333, 225.5, 28.2, 31.57142857, 85.5, 5, 31.45238095, 29, 21.10227273, 22.21428571, 28.46153846, 42.88888889, 34.69230769, 16.60869565, 17.33333333, 14.83636364, 29, 31.9, 25.5, 18.28571429, 32.82352941, 36.67391304, 19.73239437, 15.53703704, 22.61904762, 33.64285714, 36.11904762, 15.03076923, 19, 33.51724138, 25.625, 15.88888889, 28.39130435, 29.1, 39.61111111, 33.58823529, 18.47852761, 27.12903226, 29.5, 33.5, 25.15151515, 16.96190476, 22.74647887, 28.5, 35.79545455, 27.3030303, 17.69117647, 35.8, 40, 29.16666667, 20.75, 31.44444444, 24.06666667, 39.03703704, 48, 25.4, 43.8, 43.625, 39.58333333, 26.21052632, 22.53488372, 27.03448276, 24.51612903, 32.41860465, 84.5, 37.74418605, 18.9375, 30.4, 26.74025974, 28.28571429, 24.14925373, 42.375, 24.25641026, 31.39285714, 27.93103448, 26, 22.38461538, 28.86666667, 142.4545455, 26.20967742, 22.81034483, 27.25, 26.41666667, 23.57142857, 31.16666667, 37, 58.5, 20.71428571, 24.75, 41.36363636, 29.27118644, 21.74074074, 5, 25.67857143, 22, 26.15, 18.18032787, 34.125, 16.51612903],
                         'COMPLEX WORD COUNT': [1397, 1080, 1307, 1184, 1303, 967, 582, 5, 558, 1595, 1454, 931, 936, 1208, 1143, 384, 1388, 572, 506, 144, 5, 1089, 552, 1446, 494, 309, 632, 353, 314, 118, 632, 142, 533, 124, 286, 1309, 1348, 1119, 649, 1074, 352, 1189, 756, 866, 830, 810, 780, 526, 232, 583, 944, 2547, 1317, 1168, 106, 686 ,1474, 1288, 830, 1275, 725, 959, 867, 827, 137, 840 ,458, 864, 832, 120, 422, 704, 534, 811, 849, 790, 638, 1201, 1131, 648, 1338, 946, 251, 1723, 1280, 1313, 1367, 748, 1413, 650, 1024, 221, 354, 1261, 1318, 1068, 558, 259, 256, 270, 184, 181, 119, 729, 373, 1375, 477, 5, 574, 665, 832, 853, 451, 803],
                         'WORD COUNT': [1710, 1387, 1657, 1541, 1650, 1216, 715, 5, 696, 2099, 1832, 1140, 1169, 1572, 1473, 492, 1804, 705, 663, 171, 5, 1321, 696, 1857, 622, 370, 772, 451, 382, 156, 816, 174, 638, 153, 384, 1674, 1687, 1401, 839, 1425, 471, 1517, 977, 1102, 972, 1025, 1001, 653, 291, 713, 1142, 3012, 1682 ,1416, 134, 830, 1781, 1615, 1026, 1575, 901, 1203, 1074, 1040, 175, 1079, 566, 1083, 1054, 144, 508, 876, 698, 950, 996, 969, 784, 1520, 1394, 845, 1623, 1212, 304, 2059, 1584, 1618, 1695, 946, 1758, 810, 1274, 291, 433, 1567, 1625, 1323, 654, 317, 330, 374, 222, 234, 145, 891, 455, 1727, 587, 5, 719, 836, 1046, 1109, 546, 1024],
                         'SYLLABLE PER WORD': [1956, 1622, 1873, 1740, 1916, 1404, 835, 6, 791, 2385, 2214, 1267, 1383, 1741, 1700, 562, 1891, 807, 772, 199, 6, 1505, 799, 2105, 714, 412, 883, 530, 430, 175, 947, 197, 739, 172, 419, 1922, 1858, 1658, 960, 1613, 526, 1752, 1137, 1250, 1103, 1206, 1164, 745, 329, 812, 1341, 3893, 1880, 1636, 157, 988, 1995, 1872, 1166, 1861, 1049, 1336, 1165, 1264, 205, 1218, 704, 1207, 1186, 171, 593, 1021, 775, 1116, 1124, 1143, 878, 1712, 1627, 914, 1908, 1375, 367, 2366, 1709, 1922, 1936, 1131, 2048, 950, 1479, 337, 523, 1790, 1881, 1580, 747, 383, 382, 445,  282, 277, 198, 1027, 522, 1971, 656, 6, 781, 965, 1210, 1255, 607, 1136],
                         'PERSONAL PRONOUNS': [0, 3, 1, 15, 10, 12, 5, 0, 2, 6, 0, 7, 3, 11, 10, 0, 13, 0, 0, 2, 0, 2, 2, 8, 3, 0, 1, 1, 0, 1, 15, 1, 0, 4, 5, 11, 18, 13, 1, 3, 7, 6, 7, 3, 0, 7, 8, 5, 0, 0, 2, 0, 7, 0, 1, 2, 5, 4, 2, 1, 3, 31, 6, 2, 1, 11, 2, 1, 4, 0, 2, 3, 30, 0, 0, 1, 4, 2, 2, 32, 8, 2, 0, 0, 14, 6, 10, 7, 2, 2, 5, 3, 0, 2, 0, 3, 0, 4, 1, 2, 1, 5, 0, 0, 2, 0, 1, 0, 2, 2, 2, 2, 0, 7],
                         'AVG WORD LENGTH': [5.284253578732106, 4.350801479654748, 5.003737319807795, 4.520689655172414, 4.629436325678497, 4.539173789173789, 4.604790419161676, 5.333333333333333, 4.427307206068268, 4.569392033542977, 4.448961156278229, 5.179952644041042, 4.5531453362255965, 4.657668006892591, 4.767647058823529, 5.0462633451957295, 4.598625066102591, 4.892193308550186, 5.132124352331606, 5.110552763819095, 5.333333333333333, 4.785382059800664, 4.630788485607009, 4.476484560570071, 4.8277310924369745, 4.645631067961165, 4.5515288788221975, 4.839622641509434, 4.751162790697674, 5.057142857142857, 4.166842661034847, 4.538071065989848, 5.2652232746955345, 4.348837209302325, 4.353221957040573, 4.919875130072841, 4.536060279870829, 4.393244873341375, 4.179166666666666, 4.105393676379418, 4.2965779467680605, 4.789954337899544, 4.277044854881266, 4.196, 4.560290117860381, 4.844112769485903, 4.304123711340206, 4.4912751677852345, 4.541033434650456, 4.911330049261084, 4.731543624161074, 4.45723092730542, 4.7106382978723405, 5.04278728606357, 4.420382165605096, 4.3714574898785425, 4.574937343358396, 4.576923076923077, 4.6620926243567755, 4.950026867275659, 4.714013346043851, 4.457335329341317, 4.76824034334764, 5.068829113924051, 4.307317073170732, 4.446633825944171, 4.666193181818182, 4.985086992543496, 4.3903878583473865, 4.87719298245614, 4.48566610455312, 4.831537708129285, 4.059354838709678, 4.974014336917563, 4.9564056939501775, 4.866141732283465, 4.686788154897494, 4.516355140186916, 4.819913952059005, 4.166301969365427, 4.84643605870021, 4.213818181818182, 5.122615803814714, 4.831360946745562, 4.252779403159742, 4.545265348595214, 4.785640495867769, 4.510167992926614, 4.7333984375, 4.6147368421052635, 4.7417173766058145, 4.492581602373887, 4.896749521988528, 4.974301675977654, 4.818181818181818, 4.64620253164557, 5.265060240963855, 4.749347258485639, 4.227748691099476, 4.52808988764045, 4.283687943262412, 4.884476534296029, 4.505050505050505, 4.63777994157741, 4.444444444444445, 4.798579401319127, 4.836890243902439, 5.333333333333333, 4.909090909090909, 5.091191709844559, 4.821487603305785, 4.673306772908367, 5.3212520593080725, 4.695422535211268]})

start_row = writer.sheets[worksheet_name].max_row + 1
for i, row in new_data.iterrows():
    writer.sheets[worksheet_name].append(row.tolist())

writer.save()